In [ ]:
#default_exp spectrum_library.decoy_library

In [ ]:
#export
import copy
from alphabase.spectrum_library.library_base import SpecLibBase
from alphabase.io.hdf import HDF_File

class DecoyLib(SpecLibBase):
    def __init__(self, 
        target_lib:SpecLibBase,
        fix_C_term = True,
    ):
        self.__dict__ = copy.deepcopy(target_lib.__dict__)
        self.fix_C_term = fix_C_term

    def translate_to_decoy(self):
        self._decoy_seq()
        self._decoy_mod()
        self._decoy_meta()
        self._decoy_frag()

    def _decoy_meta(self):
        """
        Decoy for CCS/RT or other meta data
        """
        pass

    def _decoy_mod(self):
        """
        Decoy for modifications and modification sites
        """
        pass

    def _decoy_frag(self):
        """
        Decoy for fragment masses and intensities
        """
        self._decoy_fragment_mz()
        self._decoy_fragment_intensity()
    
    def _decoy_fragment_mz(self):
        del self._precursor_df['precursor_mz']
        del self._precursor_df['frag_start_idx']
        del self._precursor_df['frag_end_idx']

        self.calc_fragment_mz_df()
        
    def _decoy_fragment_intensity(self):
        pass

    def _decoy_seq(self):
        (
            self._precursor_df.sequence
        ) = self._precursor_df.sequence.apply(
            lambda x: (x[:-1][::-1]+x[-1])
             if self.fix_C_term else x[::-1]
        )

    def save_hdf(self, hdf_file):
        _hdf = HDF_File(
            hdf_file, 
            read_only=False, 
            truncate=True,
            delete_existing=False
        )
        _hdf.library.decoy = {
            'precursor_df': self._precursor_df,
            'fragment_mz_df': self._fragment_mz_df,
            'fragment_intensity_df': self._fragment_intensity_df,
        }

    def load_hdf(self, hdf_file):
        _hdf = HDF_File(
            hdf_file,
        )
        _hdf_lib = _hdf.library
        self._precursor_df = _hdf_lib.decoy.precursor_df.values
        self._fragment_mz_df = _hdf_lib.decoy.fragment_mz_df.values
        self._fragment_intensity_df = _hdf_lib.decoy.fragment_intensity_df.values

class DiaNNDecoyLib(DecoyLib):
    def __init__(self, 
        target_lib:SpecLibBase,
        fix_C_term = True,
        raw_AAs:str = 'GAVLIFMPWSCTYHKRQEND',
        mutated_AAs:str = 'LLLVVLLLLTSSSSLLNDQE', #DiaNN
    ):  
        super().__init__(target_lib, fix_C_term)
        self.raw_AAs = raw_AAs
        self.mutated_AAs = mutated_AAs

    def _decoy_seq(self):
        (
            self._precursor_df.sequence
        ) = self._precursor_df.sequence.apply(
            lambda x:
                x[0]+self.mutated_AAs[self.raw_AAs.index(x[1])]+
                x[2:-2]+self.mutated_AAs[self.raw_AAs.index(x[-2])]+x[-1]
        )

In [ ]:
#export
class DecoyLibProvider(object):
    def __init__(self):
        self.decoy_dict = {}

    def register(self, name, decoy_class):
        self.decoy_dict[name.lower()] = decoy_class

    def get_decoy(self, name, 
        target_lib, fix_C_term=True
    )->DecoyLib:
        return self.decoy_dict[name.lower()](
            target_lib, fix_C_term
        )

decoy_lib_provider = DecoyLibProvider()
decoy_lib_provider.register('reverse', DecoyLib)
decoy_lib_provider.register('diann', DiaNNDecoyLib)

In [ ]:
#hide
import pandas as pd
repeat = 3
peptides = ['AGHCEWQMK']*repeat
mods = ['Acetyl@Protein N-term;Carbamidomethyl@C;Oxidation@M']*repeat
sites = ['0;4;8']*repeat
peptides += ['AGHCEWQMKAADER']*repeat
mods += ['']*repeat
sites += ['']*repeat

precursor_df = pd.DataFrame({
    'sequence': peptides,
    'mods': mods,
    'mod_sites': sites
})
precursor_df['nAA'] = precursor_df['sequence'].str.len()
precursor_df['charge'] = 2
precursor_df

,sequence,mods,mod_sites,nAA,charge
0,AGHCEWQMK,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,9,2
1,AGHCEWQMK,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,9,2
2,AGHCEWQMK,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,9,2
3,AGHCEWQMKAADER,,,14,2
4,AGHCEWQMKAADER,,,14,2
5,AGHCEWQMKAADER,,,14,2


In [ ]:
#hide
target_lib = SpecLibBase(['b_z1','b_z2','y_z1','y_z2'])
target_lib._precursor_df = precursor_df
target_lib.calc_precursor_mz()
target_lib.calc_fragment_mz_df()
target_lib._fragment_intensity_df = pd.DataFrame()
import os
if not os.path.isdir('sandbox'):
    os.makedirs('sandbox')
target_lib.save_hdf('sandbox/decoy_lib.hdf')
target_lib.precursor_df

,sequence,mods,mod_sites,nAA,charge,precursor_mz,frag_start_idx,frag_end_idx
0,AGHCEWQMK,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,9,2,602.747333,0,8
1,AGHCEWQMK,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,9,2,602.747333,8,16
2,AGHCEWQMK,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,9,2,602.747333,16,24
3,AGHCEWQMKAADER,,,14,2,816.356299,24,37
4,AGHCEWQMKAADER,,,14,2,816.356299,37,50
5,AGHCEWQMKAADER,,,14,2,816.356299,50,63


In [ ]:
#hide
import numpy as np
decoy_lib = decoy_lib_provider.get_decoy('reverse', target_lib)
decoy_lib.translate_to_decoy()
decoy_lib.calc_precursor_mz()
assert np.allclose(decoy_lib.precursor_df.precursor_mz, target_lib.precursor_df.precursor_mz)

In [ ]:
#hide
decoy_lib = decoy_lib_provider.get_decoy('diann', target_lib, fix_C_term=False)
decoy_lib.translate_to_decoy()
import os
if not os.path.isdir('sandbox'):
    os.makedirs('sandbox')
decoy_lib.save_hdf('sandbox/decoy_lib.hdf')
assert len(decoy_lib.precursor_df) > 0
assert len(decoy_lib.fragment_mz_df) > 0
start,end = decoy_lib.precursor_df[['frag_start_idx','frag_end_idx']].values[0]
assert np.all(np.diff(decoy_lib.fragment_mz_df['b_z1'].values[start:end])>0)
assert np.all(np.diff(decoy_lib.fragment_mz_df['y_z1'].values[start:end])<0)

In [ ]:
#hide
_hdf = HDF_File('sandbox/decoy_lib.hdf')
assert len(_hdf.library.precursor_df.values) > 0

In [ ]:
#hide
assert len(_hdf.library.decoy.precursor_df.values) > 0

In [ ]:
#hide
test_lib = DecoyLib(target_lib)
test_lib.load_hdf('sandbox/decoy_lib.hdf')
assert len(test_lib._precursor_df) > 0

In [ ]:
#hide
os.remove('sandbox/decoy_lib.hdf')